# Nombres

## Nombres alèatoires

Ceci est un problème généralisé, depuis toujours. Est-ce que votre langage de programmation as un mot clé `rand`, qui donne une réponse entre 0 et presque 1, ou une fonction `rand()`, qui donne une réponse entre 1 et l'argument?  Est-ce que c'est en point flottant? Est-ce qu'il faut l'arrondir? le tronquer? 


Raku fournit une fonction `roll` qui sélectionne au hasard un élément d'une liste (plutôt que de générer un entier)

In [36]:
(1000..9999).roll;

1561

Un exemple du genre d'utilisation: voici un dungeon master qui veut donner des attributs au hasrd à un joueur: 

In [48]:
my @attributes = <Str Int Dex Con Cha>;

my @d6 = 1..6;

for @attributes -> $atr {
    my $stat = @d6.roll(3).sum;

    say "$atr: $stat"; 
}

Str: 14
Int: 8
Dex: 11
Con: 9
Cha: 10


Mais peut-être un dungeon master plus généreux prendrait les meilleurs 3 dés sur 5:

In [47]:
my @attributes = <Str Int Dex Con Cha>;

my @d6 = 1..6;

for @attributes -> $atr {
    my $stat = @d6
        .roll(5)
        .sort(-*)
        .head(3).sum;
        
    say "$atr: $stat"; 
}

Str: 17
Int: 12
Dex: 10
Con: 16
Cha: 15


Enfin, `roll` permet la duplication, fait la sélection avec des doubles. Mais si ont veut avoir une sélection sans doubles? 

In [57]:
my @friends = <Bob Martial Jean Freddie Bob Simon Jean>;
say @friends.pick(3);

(Bob Bob Jean)


## Précision des calculs

Le point-flottant est un problème depuis toujours. Sur STack Overflow, il y a d'innombrables questions dur le poinmt flottant. 

En mathèmathiques, l'énoncé `0.3 = 0.1+0.2`  est vrai. Mais, en ruby?

In [77]:
!ruby 'puts 0.3 == 0.1 + 0.2 ? "Yes" : "no" '

Undeclared routine:
    ruby used at line 2


Python, Perl5, C, et maintes autres langages donnent la même reponse. 

Mais, en Raku, la réponse est correcte!

In [79]:
say 0.1 + 0.2 == 0.3 ?? "yes" !! "No" ;

yes


In [81]:
say 0.3.nude;

(3 10)


In [82]:
say (1/12 + 1/5).nude;

(17 60)


In [83]:
0.3.WHAT

(Rat)

Dans Haskell, ce sont des "Rationals". Dans Raku, ce sont des Rat. (Et si les chiffres deviennent assez frands, des BigRat)

C'est plus lent que du point-flottant `hardware`, mais ça donne la bonne réponse, ce que le point flottant ne fait pas. 

Un autre exemple: 7\over 3

In [86]:
say -7/3;

-2.333333


Dans d'autres langages, on a les résultats suivants: 



# Function Signatures

## Gradual Typing

## Subsets

# Concurrency

# Parallel Processing

Le traitement en parallel est souvent difficile à gérer, et très difficile à debugger. En fait, sans passer par des processus, la majorité des langages de scripting communs aujourd'hui ont un 'Global Interpreter Lock' qui fait en sorte que on ne peut pas rouler en multi-thread. Il faut alors se pencher sur des processus sépraé, ce qui rends les choses plus difficiles encore. 

## hyper operators

Imaginons que les deux vecteurs `@row1` et `@row2` ont des millions d'éléments. 

In [87]:
my @row1 = 5, 20, 3;
my @row2 = 3, 2, 1;

# say row_add( @row1, @row2 );

[3 2 1]

Un premier essai très simple: 

In [ ]:
sub row_add( @a, @b ) {
    die if @a.elems != @b.elems;
    
    my @result;
    for 0 .. @a.end -> $i {
        push @result, @a[$i] + @b[$i];
        }
     return @result;   
    }
}

C'est sériel, un pas après l'autre. 

In [ ]:
sub row_add( @a, @b ) {
    die if @2.elems != @b.elems;
    
    return @a.map { @a[$_] + @b[$_] }, 0 .. @a.end;
}

La fonction `keys` fournit la liste d'indexes: 

In [ ]:
sub sub row_add( @a, @b ) {
    die if @2.elems != @b.elems;
    
    return @a.keys.map: { @a[$_] + @b[$_] };
}

L'opérateur Zip (semblable à la fonction zip en python):

In [ ]:
sub row_add( @a, @b ) {
    die if @2.elems != @b.elems;

    return @a Z+ @b;
}

Mais toutes ces méthodes sont en série. 

In [88]:
sub row_add( @a, @b ) {
    return @a »+« @b; # les deux longeurs doivent être identiques (exception si pas égal)
    
    #return @a <<+<< @b; retourne resultat de taille de @b, @a est réutilisé
    #return @a >>+>> @b; retourne resultat de taille de @a, @b est réutilisé
    #return @a <<+>> @b; retourne un résultat de taille du plus grand vecteur, le plus court est réutilisé
    
}

&row_add

In [89]:
say row_add( @row1, @row2 );

[8 22 4]


Les  `hyper-operators` sont potentiellement multi-thread; ils ne forcent pas le multi-thread, mais disent au compilateur que il n'y a pas d'effets secondaires, donc il peut y aller si c'est possible.

Un exemple très utile c'est quand on fait du `scaling`:

In [84]:
sub scale ( @a, $n ) { return @a »*» $n }

&scale

Une différence importante entre le meta-operateur `Z` et les hyper-opérateurs « et » est que (autres que le `multi-threading` est que `Z` ne fait qu'un niveau, tandis que « et » traversent complètement la structure. 

# Grammars

Perl était reconnu pour ses regexs. MUltiples langages de programmation on copiés ou directement implantés les regexs de perl, même s'ils ne servaient pas autant. Mais les regexs sont difficile à comprendre, même très difficile à écrire moindrement que les textes deviennent compliqués. 

Prenons, par exemple, une date iso, em perl5 on ferait:

mais, si on a besoin de lire deux dates, on fait quoi?  Prenons un fichier avec des lignes tel que: `IsoDate IsoDate Titre`

In [72]:
grammar LogLine3 {
    token year  { <.digit> ** 4 <?{ $/.Num >= 1900  }> }
    token month { <.digit> ** 2 <?{ $/.Num ~~ 1..12 }> }
    token day   { <.digit> ** 2 <?{ $/.Num ~~ 1..31 }> }
    
    token ISO-Date { <year> '-' <month> '-' <day> }
    regex book-name { \S .*? }
    token start-date { <ISO-Date> }
    token stop-date  { <ISO-Date> }
    rule TOP { ^ <start-date> <stop-date> <book-name> $ }
    }

my $line = ' 2010-02-28 2015-03-14 Harry Potter and the Methods of Rationality';

LogLine3.parse($line) or die;

say $/<book-name>.Str.perl;

"Harry Potter and the Methods of Rationality"


`Grammar` crée un `recursive-descent` parser. Pour faciliter les choses, il y a deux outils `Grammar::Tracer` et `Grammar::Debugger` pour aider à vérfier le code. 

### Palindromes

In [76]:
say (^100).grep({ $_ eq $_.flip }).elems;

19
